## 最適化問題の書き換え

$$ \min_H \|G-HF\|\_F^2+\lambda\|H\|\_1 $$
$$ \downarrow $$
$$ \min_h \|\bm{g}-F^\top \bm{h}\|\_2^2+\lambda\|\bm{h}\|\_1 $$
ただし、

$$
H=(\bm{h}_1\cdots\bm{h}_N), \;
\bm{h}=\left(\begin{matrix}
\bm{h}_1\\
\vdots\\
\bm{h}_N
\end{matrix}\right)\\
G=(\bm{g}_1\cdots\bm{g}_K), \;
\bm{g}=\left(\begin{matrix}
\bm{g}_1\\
\vdots\\
\bm{g}_K
\end{matrix}\right)
$$


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from PIL import Image
import os
import re

In [ ]:
# パラメータ設定
n = 64
m = 128
LAMBDA = 1
SEED = 5
RATIO = 0.1
DATA_PATH = '../../OneDrive - m.titech.ac.jp/Lab/data'
IMG_NAME = 'hadamard'
DIRECTORY = DATA_PATH + '/240717'
SETTING = f"{IMG_NAME}_l1_p-{int(100*RATIO)}_lmd-{int(LAMBDA)}_seed-{SEED}"
# SETTING=f"{IMG_NAME}_l1_lmd-{int(LAMBDA)}_seed-{SEED}"

In [ ]:
def matrix2vector(matrix):
    return matrix.T.flatten()


def vector2matrix(vector, s, t):
    return vector.reshape(s, t).T


def mult_mass(F, h, M):
    N = F.shape[1]
    H = h.reshape(N, M)
    res = F @ H
    return res.flatten()


def images_to_matrix(folder_path, convert_gray=True, rand=True, ratio=RATIO, seed=SEED):
    files = os.listdir(folder_path)
    files.sort(key=lambda f: int(re.search(f"{IMG_NAME}_(\d+).png", f).group(1)))
    if rand:
        random.seed(seed)
        random.shuffle(files)

    total_files = len(files)
    number_of_files_to_load = int(total_files * ratio)
    selected_files = files[:number_of_files_to_load]
    selected_files.sort(key=lambda f: int(re.search(f"{IMG_NAME}_(\d+).png", f).group(1)))

    images = []
    use_list = []

    for file in selected_files:
        index = int(re.sub(r'\D', '', file))
        use_list.append(index)
        img = Image.open(os.path.join(folder_path, file))
        if convert_gray:
            img = img.convert('L')
        img_array = np.asarray(img).flatten()
        img_array = img_array / 255
        images.append(img_array)

    return np.column_stack(images), use_list


def images_to_matrix_with_use_list(folder_path, use_list, convert_gray=True):
    files = os.listdir(folder_path)
    files.sort(key=lambda f: int(re.search(f"{IMG_NAME}_(\d+).png", f).group(1)))

    images = []

    for index in use_list:
        file = f"{IMG_NAME}_{index}.png"
        if file in files:
            img = Image.open(os.path.join(folder_path, file))
            if convert_gray:
                img = img.convert('L')
            img_array = np.asarray(img).flatten()
            img_array = img_array / 255
            images.append(img_array)
        else:
            print(f"Warning: File {file} not found in {folder_path}")

    return np.column_stack(images)


def display_images(index_array, folder_path, images_per_page=64, white=None):
    num_images = len(index_array)
    num_pages = math.ceil(num_images / images_per_page)
    combined_image = None

    for page in range(num_pages):
        start_index = page * images_per_page
        end_index = min(start_index + images_per_page, num_images)
        page_indices = index_array[start_index:end_index]

        num_page_images = len(page_indices)
        grid_size = math.ceil(math.sqrt(num_page_images))

        fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))
        axes = axes.flatten()

        for i, idx in enumerate(page_indices):
            image_path = f"{folder_path}/{IMG_NAME}_{idx}.png"
            img = Image.open(image_path)
            img = img.convert('L')
            img = np.asarray(img) / 255
            if white is not None:
                img = 2 * img - white
            if combined_image is None:
                combined_image = img
            else:
                combined_image = combined_image + img
            axes[i].imshow(img, cmap='gray')
            axes[i].set_title(f"{idx}")
            axes[i].axis('off')

        for i in range(num_page_images, len(axes)):
            axes[i].axis('off')

        plt.tight_layout()
        plt.show()

    if combined_image is not None:
        plt.figure(figsize=(10, 8))
        sns.heatmap(combined_image, annot=False, cmap='gray')
        plt.title('Combined Image')
        plt.axis('off')
        plt.show()

    return combined_image

In [ ]:
def fista(F, g, lmd, prox, max_iter=500, tol=1e-2):
    """
    Solve the optimization problem using FISTA:
    min_h ||g - Fh||_2^2 + lambda * ||h||_1

    Parameters:
    - F: numpy array, the matrix F
    - g: numpy array, the vector g
    - lmd: float, the regularization parameter
    - max_iter: int, the maximum number of iterations
    - tol: float, the tolerance for convergence

    Returns:
    - h: numpy array, the solution vector h
    """
    # Initialize variables
    n = F.shape[1]
    t = 1
    h = np.zeros(m * m * n)
    y = np.zeros(m * m * n)

    # Lipschitz constant
    L = np.linalg.norm(F.T @ F, ord=2) * 2
    # L = 4096 * 2
    print(f"L: {L}")
    gamma = 1 / L

    for i in range(max_iter):
        grad = mult_mass(F.T, (mult_mass(F, y, m * m) - g), m * m)
        h_new = prox(y - gamma * grad, gamma * lmd)
        t_new = (1 + np.sqrt(1 + 4 * t**2)) / 2
        y_new = h_new + (t - 1) / t_new * (h_new - h)

        error = np.linalg.norm(y_new - y)
        print(f"iter: {i}, error: {error}")
        if error < tol:
            break

        t = t_new.copy()
        h = h_new.copy()
        y = y_new.copy()

    return y_new


def prox_l1(u, tau):
    """Soft thresholding operator for L1 norm."""
    return np.sign(u) * np.maximum(np.abs(u) - tau, 0)

In [ ]:
def elements_in_A_not_in_B(A, B):
    set_B = set(B)
    return [element for element in A if element not in set_B]


def list_to_comma_separated_string(lst):
    return ', '.join(map(str, lst))


def common_elements(A, B):
    set_A = set(A)
    set_B = set(B)
    common = set_A.intersection(set_B)
    return sorted(common)


def union_elements(A, B):
    set_A = set(A)
    set_B = set(B)
    union = set_A.union(set_B)
    return sorted(union)


def remove_elements_in_B_from_A(A, B):
    set_B = set(B)
    return [element for element in A if element not in set_B]

In [ ]:
# G, use2 = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/", seed=2)
# G, use5 = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/", seed=5)
# G, use3 = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/", seed=3)
# common_2_5 = common_elements(use2, use5)
# common3_2_5 = union_elements(common_2_5[1:], use3)
# print(list_to_comma_separated_string(common_2_5))
# print(len(common_2_5))
# print(list_to_comma_separated_string(common3_2_5))
# print(len(common3_2_5))
useAll = list(range(1, 4097))
useAll.remove(32)

In [ ]:
# load images
G, use = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/")
# G = images_to_matrix_with_use_list(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/", useAll)
F, _ = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_input/")
# F = images_to_matrix_with_use_list(f"{DATA_PATH}/{IMG_NAME}{n}_input/", useAll)
print("K=", F.shape[1])
white_img = Image.open(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/{IMG_NAME}_1.png")
white_img = white_img.convert('L')
white_img = np.asarray(white_img) / 255
white = white_img.flatten()
white = white[:, np.newaxis]
H1 = np.tile(white, F.shape[1])
F_hat = 2 * F - 1
G_hat = 2 * G - H1

g = matrix2vector(G_hat)

In [ ]:
h = fista(F_hat.T, g, LAMBDA, prox_l1)

In [ ]:
if not os.path.exists(DIRECTORY):
    os.makedirs(DIRECTORY)
if not os.path.exists(DIRECTORY + '/systemMatrix'):
    os.makedirs(DIRECTORY + '/systemMatrix')

In [ ]:
H = vector2matrix(h, n**2, m**2)
np.save(f"{DIRECTORY}/systemMatrix/H_matrix_{SETTING}.npy", H)

SAMPLE_NAME = 'Cameraman64'
sample_image = Image.open(f"{DATA_PATH}/sample_image64/{SAMPLE_NAME}.png").convert('L')
sample_image = np.asarray(sample_image).flatten() / 255

Hf = H @ sample_image
Hf_img = Hf.reshape(m, m)
Hf_img = np.clip(Hf_img, 0, 1)
Hf_pil = Image.fromarray((Hf_img * 255).astype(np.uint8), mode='L')

FILENAME = f"{SAMPLE_NAME}_{SETTING}.png"
fig, ax = plt.subplots(figsize=Hf_img.shape[::-1], dpi=1, tight_layout=True)
ax.imshow(Hf_pil, cmap='gray')
ax.axis('off')
fig.savefig(f"{DIRECTORY}/{FILENAME}", dpi=1)
plt.show()

In [ ]:
H_true = np.load(f"{DATA_PATH}/systemMatrix/H_matrix_true.npy")
rem = np.linalg.norm(H_true - H, 'fro')
print(rem)

In [ ]:
# 真のHを計算
G_full = images_to_matrix_with_use_list(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/", list(range(1, 4097)))
F_full = images_to_matrix_with_use_list(f"{DATA_PATH}/{IMG_NAME}{n}_input/", list(range(1, 4097)))
white_img = Image.open(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/{IMG_NAME}_1.png").convert('L')
white = np.asarray(white_img).flatten() / 255
white = white[:, np.newaxis]
H1 = np.tile(white, F_full.shape[1])
F_hat_full = 2 * F_full - 1
G_hat_full = 2 * G_full - H1
H_true = G_hat_full @ F_hat_full.T / (n**2)
np.save(f"{DATA_PATH}/systemMatrix/H_matrix_true.npy", H_true)